In [1]:
import pandas as pd
import networkx as nx
from modules import ps
import sys
sys.path.append("C:/Users/kubic/Desktop/machine learning/Polarization/")

import functions

import utils
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
import umap
import numpy as np
from itertools import combinations
from scipy.stats import gaussian_kde

c:\Users\kubic\OneDrive\Dokumenty\envs\polarization\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def make_all_edges(df):
    df = df.groupby(by = ["rollnumber", "cast_code"]).apply(lambda x: pd.DataFrame(list(combinations(x["icpsr"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value
    df.columns = ("src", "trg")
    df = df.groupby(by = ["src", "trg"]).size().reset_index().rename(columns = {0: "nij"})                        # Counts how many times a pair of congressmen appears in df (i.e. they co-voted)
    return df

def make_pdfs(edges, nodes):
    party_lookup = nodes.set_index("icpsr")["party_code"].to_dict()
    edges["party_src"] = edges["src"].map(party_lookup)
    edges["party_trg"] = edges["trg"].map(party_lookup)
    edges["same_party"] = edges["party_src"] == edges["party_trg"]
    edges["nij"] /= edges["nij"].max()  # Normalize co-vote counts

    sp_pdf = gaussian_kde(edges[edges["same_party"]]["nij"])
    cp_pdf = gaussian_kde(edges[~edges["same_party"]]["nij"])
    return edges, sp_pdf, cp_pdf
def find_intersection(kde1, kde2, init_interval=0.01, scope=[0.4,1], convergence=0.0001):
    x_left, x_right = scope[0], scope[0] + init_interval
    while x_right < scope[1]:
        left, right = kde1(x_left)[0] - kde2(x_left)[0], kde1(x_right)[0] - kde2(x_right)[0]
        if left * right < 0:
            if init_interval <= convergence:
                return x_right
            return find_intersection(kde1, kde2, init_interval / 10, [x_left, x_right])
        x_left, x_right = x_right, x_right + init_interval
    return scope[0]

In [3]:

congresses = ['095', '096', '097','098', '099', '100', '101', '102', '103','104',
              '105', '106', '107','108', '109', '110', '111', '112','113', '114',
              '115', '116','117','118']

# Create Images directory if it doesn't exist
os.makedirs("Images", exist_ok=True)

for congress in congresses:
    input_votes = f"Data/USA/Filtered/H{congress}_filtered_USA_votes.csv"
    try:
        votes_df = pd.read_csv(input_votes)
        edges_df = make_all_edges(votes_df)
        edges_df, sp_pdf, cp_pdf = make_pdfs(edges_df, votes_df)

        x_values = np.linspace(0, 1, 1000)
        y_sp = sp_pdf(x_values)
        y_cp = cp_pdf(x_values)
        y_intersection = np.minimum(y_sp, y_cp)

        plt.figure(figsize=(8, 5))
        plt.plot(x_values, y_sp, label="Same-party", color="black")
        plt.plot(x_values, y_cp, label="Cross-party", color="gray")
        plt.xlabel("Normalized co-vote count")
        plt.ylabel("Density")
        plt.title(f"Density distributions of co-vote counts by party type for {congress}th Congress")
        plt.legend()
        plt.tight_layout()

        output_path = f"Images/Congress_{congress}_density_plot.png"
        plt.savefig(output_path)
        plt.close()

        print(f"Saved: {output_path}")
    except Exception as e:
        print(f"Error processing Congress {congress}: {e}")

C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\694759260.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["rollnumber", "cast_code"]).apply(lambda x: pd.DataFrame(list(combinations(x["icpsr"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Congress_095_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\694759260.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["rollnumber", "cast_code"]).apply(lambda x: pd.DataFrame(list(combinations(x["icpsr"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Congress_096_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\694759260.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["rollnumber", "cast_code"]).apply(lambda x: pd.DataFrame(list(combinations(x["icpsr"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


...

Saved: Images/Congress_116_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\694759260.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["rollnumber", "cast_code"]).apply(lambda x: pd.DataFrame(list(combinations(x["icpsr"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Congress_117_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\694759260.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["rollnumber", "cast_code"]).apply(lambda x: pd.DataFrame(list(combinations(x["icpsr"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Congress_118_density_plot.png


In [ ]:
def save_network(edges, congress, threshold, output_folder):
    edges = edges[edges["nij"] > threshold]
    edges = edges[["src", "trg"]].astype(int)
    
    edges_output = os.path.join(output_folder, f"congress{congress}_edges.csv")
    edges.to_csv(edges_output, sep=",", index=False, header=False)
    
    print(f"Network saved: {edges_output}")

def process_congresses(congress_list, input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for congress in congress_list:
        print(f"Processing Congress {congress}...")
        input_votes = os.path.join(input_folder, f"H{congress}_filtered_USA_votes.csv")
        
        if not os.path.exists(input_votes):
            print(f"Warning: Data file for Congress {congress} not found, skipping.")
            continue

        # Load data
        votes_df = pd.read_csv(input_votes)

        # Create edgelist
        edges_df = make_all_edges(votes_df)

        # Generate PDFs
        edges_df, sp_pdf, cp_pdf = make_pdfs(edges_df, votes_df)

        # Compute intersection (threshold)
        threshold = find_intersection(sp_pdf, cp_pdf)

        # Save network
        save_network(edges_df, congress, threshold, output_folder)

# List of congress numbers
congresses = ['097']

# Run the process
input_folder = "Data/USA/Filtered/"
output_folder = "Data/USA/Micheles/"

process_congresses(congresses, input_folder, output_folder)

In [ ]:
def calc_pol(congress, data_path="data/USA/Raw/", edge_list_folder="Data/USA/Micheles/"):
    edge_list_file = os.path.join(edge_list_folder, f"congress{congress}_edges.csv")
    
    if not os.path.exists(edge_list_file):
        print(f"Warning: Edge list for Congress {congress} not found, skipping.")
        return None  # Skip if no edge list exists

    edge_df = pd.read_csv(edge_list_file, header=None, names=['Source', 'Target'])

    # Create the graph
    G = nx.from_pandas_edgelist(edge_df, 'Source', 'Target')

    # Convert nodes to integers
    G = nx.relabel_nodes(G, lambda x: int(x))

    # Load members' data
    members_file = os.path.join(data_path, f"H{congress}_members.csv")
    if not os.path.exists(members_file):
        print(f"Warning: Members file for Congress {congress} not found, skipping.")
        return None

    members_df = pd.read_csv(members_file).dropna(subset=["nominate_dim1"])
    members_df["icpsr"] = members_df["icpsr"].astype(int)

    # Create dictionary of opinions
    opinions_x = dict(zip(members_df["icpsr"], members_df["nominate_dim1"]))

    # Filter only existing nodes in the graph
    opinions = {node: opinions_x[node] for node in G.nodes if node in opinions_x}

    if not opinions:  # If no valid opinions exist, skip
        print(f"Warning: No valid opinions found for Congress {congress}, skipping.")
        return None

    # Normalize opinions between -1 and 1
    min_opinion, max_opinion = min(opinions.values()), max(opinions.values())
    opinions = {k: 2 * (v - min_opinion) / (max_opinion - min_opinion) - 1 for k, v in opinions.items()}

    # Compute polarization score
    pol_score = ps.ge(opinions, {}, G)
    
    return pol_score
congresses =  ['095', '096', '097','098', '099', '100', '101', '102', '103','104', '105', '106', '107','108', '109', '110', '111', '112','113', '114', '115', '116','117','118']


pol_scores = {}
for congress in congresses:
    pol_score = calc_pol(congress)  # Run function
    pol_scores[int(congress)] = pol_score  
    print(f"Congress {int(congress)}: Polarization Score = {pol_score}")


plt.figure(figsize=(10, 5))
plt.plot(pol_scores.keys(), pol_scores.values(), marker='o', linestyle='-', color='b', label="Polarization Score")
plt.xlabel("Congress")
plt.ylabel("Polarization Score")
plt.title("Polarization Score by Congress")
plt.xticks(list(pol_scores.keys()))  # Set x-axis labels to be congress numbers
plt.legend()
plt.grid(True)
plt.show()

In [7]:
party_colors = {
    100: "blue",   # Democrat
    200: "red"     # Republican
}

congresses = ['095', '096', '097','098', '099', '100', '101', '102', '103','104',
              '105', '106', '107','108', '109', '110', '111', '112','113', '114',
              '115', '116','117','118']

# Create output folder
os.makedirs("Images/networks_paper", exist_ok=True)

for congress in congresses:
    # Load edge list
    edge_path = f"Data/USA/Micheles/congress{congress}_edges.csv"
    edges_df = pd.read_csv(edge_path)

    # Load node attributes (party_code)
    node_path = f"Data/USA/Filtered/H{congress}_filtered_USA_votes.csv"
    nodes_df = pd.read_csv(node_path)

    # Build graph
    G = nx.from_pandas_edgelist(edges_df, source='Source', target='Target', edge_attr=True)

    # Map party codes to colors
    party_dict = nodes_df.set_index("id")["party_code"].to_dict()
    color_map = [party_colors.get(party_dict.get(node, None), "gray") for node in G.nodes()]

    # Generate layout
    pos = nx.spring_layout(G, seed=42)

    # Draw the network
    plt.figure(figsize=(10, 10))
    nx.draw_networkx_nodes(G, pos, node_color=color_map, node_size=30)
    nx.draw_networkx_edges(G, pos, alpha=0.3, width=0.5)
    plt.title(f"Congress {congress} Network Graph")
    plt.axis("off")

    # Save plot
    out_path = f"Images/networks_paper/congress_{congress}_network.png"
    plt.savefig(out_path, dpi=300, bbox_inches='tight')
    plt.close()

NetworkXError: Invalid edge_attr argument: No columns found with name: []

## Denmark

In [4]:
def make_all_edges(df):
    df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value
    df.columns = ("src", "trg")
    df = df.groupby(by = ["src", "trg"]).size().reset_index().rename(columns = {0: "nij"})                        # Counts how many times a pair of congressmen appears in df (i.e. they co-voted)
    return df

def make_pdfs(edges, nodes):
    party_lookup = nodes.set_index("aktørid")["party"].to_dict()
    edges["party_src"] = edges["src"].map(party_lookup)
    edges["party_trg"] = edges["trg"].map(party_lookup)
    edges["same_party"] = edges["party_src"] == edges["party_trg"]
    edges["nij"] /= edges["nij"].max()  # Normalize co-vote counts

    sp_pdf = gaussian_kde(edges[edges["same_party"]]["nij"])
    cp_pdf = gaussian_kde(edges[~edges["same_party"]]["nij"])
    return edges, sp_pdf, cp_pdf

def find_intersection(kde1, kde2, init_interval=0.01, scope=[0.4,1], convergence=0.0001):
    x_left, x_right = scope[0], scope[0] + init_interval
    while x_right < scope[1]:
        left, right = kde1(x_left)[0] - kde2(x_left)[0], kde1(x_right)[0] - kde2(x_right)[0]
        if left * right < 0:
            if init_interval <= convergence:
                return x_right
            return find_intersection(kde1, kde2, init_interval / 10, [x_left, x_right])
        x_left, x_right = x_right, x_right + init_interval
    return scope[0]

In [5]:
congresses = ['01_05','05_07','07_11','11_15','15_19','19_22']
for congress in congresses:
    input_votes = f"Data/Denmark/Raw/P{congress}_DK.csv"
    try:
        votes_df = pd.read_csv(input_votes)
        edges_df = make_all_edges(votes_df)
        edges_df, sp_pdf, cp_pdf = make_pdfs(edges_df, votes_df)

        x_values = np.linspace(0, 1, 1000)
        y_sp = sp_pdf(x_values)
        y_cp = cp_pdf(x_values)
        y_intersection = np.minimum(y_sp, y_cp)

        plt.figure(figsize=(8, 5))
        plt.plot(x_values, y_sp, label="Same-party", color="black")
        plt.plot(x_values, y_cp, label="Cross-party", color="gray")
        plt.xlabel("Normalized co-vote count")
        plt.ylabel("Density")
        plt.title(f"Density distributions of co-vote counts by party type for {congress}th Congress Denmark")
        plt.legend()
        plt.tight_layout()

        output_path = f"Images/Denmark_Congress_{congress}_density_plot.png"
        plt.savefig(output_path)
        plt.close()

        print(f"Saved: {output_path}")
    except Exception as e:
        print(f"Error processing Congress {congress}: {e}")

C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\1134079950.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Denmark_Congress_01_05_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\1134079950.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Denmark_Congress_05_07_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\1134079950.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


...

Saved: Images/Denmark_Congress_11_15_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\1134079950.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Denmark_Congress_15_19_density_plot.png


C:\Users\kubic\AppData\Local\Temp\ipykernel_5500\1134079950.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Saved: Images/Denmark_Congress_19_22_density_plot.png


In [ ]:
def save_network(edges, congress, threshold, output_folder):
    edges = edges[edges["nij"] > threshold]
    edges = edges[["src", "trg"]].astype(int)
    
    edges_output = os.path.join(output_folder, f"congress_{congress}_edges.csv")
    edges.to_csv(edges_output, sep=",", index=False, header=False)
    
    print(f"Network saved: {edges_output}")

def process_congresses(congress_list, input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for congress in congress_list:
        print(f"Processing Congress {congress}...")
        input_votes = os.path.join(input_folder, f"P{congress}_DK.csv")
        
        if not os.path.exists(input_votes):
            print(f"Warning: Data file for Congress {congress} not found, skipping.")
            continue

        # Load data
        votes_df = pd.read_csv(input_votes)

        # Create edgelist
        edges_df = make_all_edges(votes_df)

        # Generate PDFs
        edges_df, sp_pdf, cp_pdf = make_pdfs(edges_df, votes_df)

        # Compute intersection (threshold)
        threshold = find_intersection(sp_pdf, cp_pdf)

        # Save network
        save_network(edges_df, congress, threshold, output_folder)

# List of congress numbers
congresses = ['01_05','05_07','07_11','11_15','15_19','19_22']

# Run the process
input_folder = "Data/Denmark/Raw"
output_folder = "Data/Denmark/Micheles/"

process_congresses(congresses, input_folder, output_folder)

Processing Congress 01_05...


C:\Users\kubic\AppData\Local\Temp\ipykernel_19252\3087672235.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Network saved: Data/Denmark/Micheles/congress_01_05_edges.csv
Processing Congress 05_07...


C:\Users\kubic\AppData\Local\Temp\ipykernel_19252\3087672235.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Network saved: Data/Denmark/Micheles/congress_05_07_edges.csv
Processing Congress 07_11...


...

Network saved: Data/Denmark/Micheles/congress_11_15_edges.csv
Processing Congress 15_19...


C:\Users\kubic\AppData\Local\Temp\ipykernel_19252\3087672235.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Network saved: Data/Denmark/Micheles/congress_15_19_edges.csv
Processing Congress 19_22...


C:\Users\kubic\AppData\Local\Temp\ipykernel_19252\3087672235.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(by = ["afstemningid", "typeid_x"]).apply(lambda x: pd.DataFrame(list(combinations(x["aktørid"], 2)))) # "combinations" makes all possible pairs of icpsr codes for every vote value


Network saved: Data/Denmark/Micheles/congress_19_22_edges.csv
